In [15]:
'''
A (hopefully) Simple API for serving explanation score requests.

input_string = (
    f"{question} answer: {gold_label}. "
    + f" explanation: {abstr_expl}."
)

here are some example input strings:

If you feel like everything is spinning while climbing you are experiencing what? answer: vertigo. explanation: Vertigo is often experienced while climbing or at heights.
Where do you get clothes in a shopping bag? answer: retail store. explanation: For any large item where convenience is beneficial, one might go to a retail store, either a regular one or a big-box store like walmart.
Where should a cat be in a house? answer: floor. explanation: A cat should be on the floor, not on a rug.
'''
import pdb
import argparse
import torch
import transformers
import os
import tqdm
import numpy as np

_model, _tokenizer = None, None

model2url = {
    'large': 'https://storage.googleapis.com/ai2-mosaic-public/projects/few-shot-explanations/pretrained_models/commonsense_qa/valloss%3D0.28665~model%3Dt5-large~lr%3D0.0001~seed%3D1~labelagg%3D0_just_weights.pt',
    '3b': 'https://storage.googleapis.com/ai2-mosaic-public/projects/few-shot-explanations/pretrained_models/commonsense_qa/valloss%3D0.28925~model%3Dt5-3b~lr%3D0.0001~seed%3D1~labelagg%3D0_just_weights.pt',
    '11b': 'https://storage.googleapis.com/ai2-mosaic-public/projects/few-shot-explanations/pretrained_models/commonsense_qa/cose_deepspeed_valloss%3D0.00000~model%3Dt5-11b~lr%3D0.00001~seed%3D1~labelagg%3D0.pt',
}

def get_model(model_type, device=None):
    global _model, model2url
    if model_type not in {'11b', '3b', 'large'}:
        raise NotImplementedError('{} is not a valid model please use "3b" or "large"'.format(model_type))

    if _model is None:
        hf_model_name = 't5-' + model_type
        print('Loading model: this will run only once.')

        if model_type == 'large':
            model_path = 'csqa_models/t5-large.pt'
        elif model_type == '3b':
            model_path = 'csqa_models/valloss=0.28925~model=t5-3b~lr=0.0001~seed=1~labelagg=0_just_weights.pt'
        elif model_type == '11b':
            model_path = 'csqa_models/cose_deepspeed_valloss=0.00000~model=t5-11b~lr=0.00001~seed=1~labelagg=0.pt'

        if not os.path.exists(model_path):
            print('Please download weights for {} model and put in current directory.'.format(model_path))
            print('for example, wget {}'.format(model2url[model_type]))
            quit()

        state = torch.load(model_path)
        if 'model_state_dict' in state:
            state = state['model_state_dict']

        _model = transformers.AutoModelForSeq2SeqLM.from_pretrained(hf_model_name)
        if model_type == '11b': # need to resize due to deepspeed, these entires are not accessed.
            _model.resize_token_embeddings(len(transformers.AutoTokenizer.from_pretrained(hf_model_name)))
        _model.load_state_dict(state)
        _model.eval()
        if device is not None:
            _model = _model.to(device)

    return _model


def get_tokenizer(model_type):
    global _tokenizer
    if model_type not in {'3b', 'large', '11b'}:
        raise NotImplementedError('{} is not a valid model please use "3b" or "large" or "11b"'.format(model_type))

    if _tokenizer is None:
        hf_model_name = 't5-' + model_type
        _tokenizer = transformers.T5TokenizerFast.from_pretrained(hf_model_name)

    return _tokenizer


class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        res = self.tokenizer(self.data[idx]['input'], truncation=True)
        res['labels'] = self.tokenizer(self.data[idx]['label']).input_ids
        return res

    def __len__(self):
        return len(self.data)


def get_scores(inputs, model_type, device=None, batch_size=32, verbose=False):
    '''
    Inputs:
      - a list of explanations to score, e.g.,:
        premise: A man getting a tattoo on his back. hypothesis: A woman is getting a tattoo. answer: contradiction. explanation: Because the tattoo artist is a man, the person getting the tattoo is not a woman.
      - model type, either "3b" or "large" or "11b"
      - device: which torch device to load model on, e.g., "cuda:3"
    Outputs:
      - P(good explanation); higher is better
    '''
    assert model_type in {'large', '3b', '11b'}

    if isinstance(inputs, str):
        inputs = [inputs]

    model = get_model(model_type, device=device)
    tokenizer = get_tokenizer(model_type)

    score_itr = T5Dataset([{'input': inp, 'label': 'x'} for inp in inputs], tokenizer) # dummy labels for inference
    data_collator = transformers.DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=-100,
#         return_tensors='pt'
    )
    score_itr = torch.utils.data.DataLoader(score_itr, shuffle=False, collate_fn=data_collator, batch_size=batch_size)
    score_itr = score_itr if not verbose else tqdm.tqdm(score_itr, total=len(score_itr))

    good_idx, bad_idx = tokenizer('good').input_ids[0], tokenizer('bad').input_ids[0]
    scores = []
    
    with torch.no_grad():
        for batch in score_itr:
            if device is not None:
                input_ids, attention_mask, targets = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            model_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            logits_pos = model_output['logits'][:, 0, good_idx].cpu().numpy()
            logits_neg = model_output['logits'][:, 0, bad_idx].cpu().numpy()
            exp_logit_pos, exp_logit_neg = np.exp(logits_pos), np.exp(logits_neg)
            score = list([float(x) for x in exp_logit_pos / (exp_logit_pos + exp_logit_neg)])
            #pdb.set_trace()
            scores.extend(score)
    return scores


# def parse_args():
#     '''
#     Optional args for main function, mostly just to test.
#     '''
#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         'model_type',
#         default='large',
#         choices={'large', '3b', '11b'})
#     parser.add_argument(
#         '--batch_size',
#         default=32,
#         type=int)

#     args = parser.parse_args(['--batch_size', '1'])
#     return args

In [16]:
# args = parse_args()
# parser = argparse.ArgumentParser()
# parser.add_argument(
#     'model_type',
#     default='large',
#     choices={'large', '3b', '11b'})
# parser.add_argument(
#     '--batch_size',
#     default=32,
#     type=int)

# args = parser.parse_args(["--model_type", "3b"])
# args.device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'
np.random.seed(1)
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# scores = get_scores(
#     ['If you feel like everything is spinning while climbing you are experiencing what? answer: vertigo. explanation: Vertigo is often experienced while climbing or at heights.',
#      'Where do you get clothes in a shopping bag? answer: retail store. explanation: For any large item where convenience is beneficial, one might go to a retail store, either a regular one or a big-box store like walmart.',
#      'Where should a cat be in a house? answer: floor. explanation: A cat should be on the floor, not on a rug.'],
#     'large',
#     device='cuda:0',
#     batch_size=1,
#     verbose=False)
# print(scores)


In [17]:
import json
from tqdm import tqdm
with open("../../scripts/results/dev_rationale_pair.json") as f:
    rationale_pair_dev_data = json.load(f)
    
    

In [18]:
rationale_pair_dev_data[0].keys(), len(rationale_pair_dev_data)

(dict_keys(['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'our_explanation', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask', 'question_encoding', 'common_expl_list', 'generated_explanation']),
 986)

In [19]:
from tqdm import tqdm
qae_list = []
score_list = []
for da in tqdm(rationale_pair_dev_data, total=len(rationale_pair_dev_data)):
    qae = "{} answer: {} explanation: {}".format(da['question'], 
                                                 da['answer'], 
                                                 da['generated_explanation'])

    scores = get_scores(
        [qae],
        'large',
        device='cuda:0',
        batch_size=1,
        verbose=False)
    score_list.append(scores[0])
    if scores[0] > 0.7 or scores[0] < 0.2:
        print("question: {}".format(da['question']))
        print("answer: {}".format(da['answer']))
        print("common_expl_list: {}".format(da['common_expl_list']))
        print("generated_explanation: {}".format(da['generated_explanation']))
        print("score: {}".format(scores[0]))

  0%|                                                                                                                                                                               | 0/986 [00:00<?, ?it/s]

Loading model: this will run only once.


  0%|▌                                                                                                                                                                    | 3/986 [00:26<1:51:30,  6.81s/it]

question: Where can you likely buy many poems?
answer: book store
common_expl_list: ['A bookstore sells a variety of books, including poetry books; chains of bookstores sometimes specialize in categories such as poetry or literature.', 'A book store is a place where you can buy books, including poetry books.']
generated_explanation: <pad> <extra_id_0> Books are a common way to collect poetry, and book stores often sell poetry collections.<extra_id_1> </s>
score: 0.7897491455078125


  1%|█▌                                                                                                                                                                     | 9/986 [00:26<20:05,  1.23s/it]

...

question: The man was going fishing instead of work, what is he seeking?
answer: relaxation
common_expl_list: ['The man wants to make himself feel less stressed from work. Fishing is a relaxing, stress-relieving activity.', 'Fishing is a relaxing activity.', 'A man seeking relaxation does so by doing something enjoyable, e.g., taking a fishing trip.']
generated_explanation: <pad> <extra_id_0> Fishing is a great way to relax, especially if you're working at a desk all day. You can relax by being outside, and fishing can provide that kind of relaxation.<extra_id_1> </s>
score: 0.7517766356468201
question: What would you need if you want to smoke?
answer: cigarette
common_expl_list: ['Cigarettes are the most common form of tobacco smoking.']
generated_explanation: <pad> <extra_id_0> Cigarettes are lighted sticks of tobacco that are rolled up and smoked.<extra_id_1> </s>
score: 0.7065684795379639
question: Where is a likely place to store unused soap?
answer: cabinet
common_expl_list: ['So

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 980/986 [01:17<00:00, 20.33it/s]

question: The detective was finding information from witnesses, why would he do that?
answer: get answers
common_expl_list: ['A detective role, and a job of other cops and detectives, is to get answers; thus, one would need to talk to witnesses to do this.', 'A detective is trying to solve a crime, and so he would want to get answers from witnesses.']
generated_explanation: <pad> <extra_id_0> Detectives are trained to find information from witnesses, and one of the best ways to find information from witnesses is to talk to them.<extra_id_1> </s>
score: 0.7269184589385986
question: If you want to kill someone you can do what to them with a gun?
answer: shoot
common_expl_list: ['Shooting someone is a common way they can be killed, as a bullet from a gun could penetrate their skin and kill them directly.', 'If you shoot someone with a gun, they will die.']
generated_explanation: <pad> <extra_id_0> Guns can be used to kill people by shooting them.<extra_id_1> </s>
score: 0.7199507355690002

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 986/986 [01:17<00:00, 12.74it/s]

question: What is main benefit to exercising?
answer: get in shape
common_expl_list: ['From a health perspective, exercising is healthy. It increases flexibility, circulation, improves mood, burns calories, strengthens muscles, helps us sleep better, etc.', 'To get in shape, work out. Exercising can help you become stronger or thinner. Exercising is sometimes done to strengthen the heart or lungs, or sweat.', "Exercise is a way to get in shape, which is a way to improve one's physical health.", 'One major reason to exercise is to get healthier by becoming in shape.']
generated_explanation: <pad> <extra_id_0> Exercise is a great way to get in shape, and getting in shape has many benefits.<extra_id_1> </s>
score: 0.8633697628974915
question: What can you use to store a book while traveling?
answer: suitcase
common_expl_list: ['People can store books in their suitcases, especially if they are on the move.']
generated_explanation: <pad> <extra_id_0> A suitcase is a container that holds ite

## evaluate generated rationale with bert-score

In [3]:
import datasets

bertscore_metric = datasets.load_metric("bertscore")

In [10]:
import pdb
bertscores = []
for da in tqdm(rationale_pair_dev_data, total=len(rationale_pair_dev_data)):
    generated_expl = da['generated_explanation']
    common_expl_list = da['common_expl_list']
    pred_expl = generated_expl.split("<extra_id_0> ")[1].split("<extra_id_1>")[0]
    list_gold_expl = common_expl_list
    #print(generated_expl)
    #print(generated_expl.split("<extra_id_0> ")[1].split("<extra_id_1>")[0])
    instance_bertscores = []
    for gold_expl in list_gold_expl: 
        score = bertscore_metric.compute(predictions=[pred_expl.lower()], references=[gold_expl.lower()], lang="en")["f1"][0]*100
        instance_bertscores.append(score)
    bertscores.append(max(instance_bertscores))
bertscore = np.mean(bertscores)
    #pdb.set_trace()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 986/986 [01:23<00:00, 11.77it/s]


NameError: name 'np' is not defined

In [11]:

import numpy as np
bertscore = np.mean(bertscores)


In [12]:
bertscore

89.63499489952537

In [13]:
bertscores

[89.01979327201843,
 90.37740230560303,
 85.25624871253967,
 93.10430884361267,
 87.44266033172607,
 87.51632571220398,
 91.12799167633057,
 91.29958748817444,
 88.49538564682007,
 89.39582109451294,
 90.31085968017578,
 90.40752053260803,
 87.21712827682495,
 93.52872371673584,
 88.200443983078,
 91.08121991157532,
 88.90480399131775,
 85.6385886669159,
 89.80599641799927,
 87.95192837715149,
 89.95508551597595,
 86.57296895980835,
 90.90205430984497,
 90.48690795898438,
 89.27825689315796,
 89.53070044517517,
 90.07056951522827,
 88.45992684364319,
 87.71679401397705,
 92.56680011749268,
 93.09734106063843,
 88.12485337257385,
 85.48378348350525,
 87.04838156700134,
 88.9463484287262,
 89.0618622303009,
 88.52788805961609,
 90.0373101234436,
 91.38297438621521,
 89.05656337738037,
 90.01131057739258,
 89.20090794563293,
 90.45494794845581,
 87.87121176719666,
 86.95672154426575,
 88.81932497024536,
 87.67191767692566,
 88.77227306365967,
 89.5970106124878,
 96.81265354156494,
 89.077

In [21]:
np.mean(score_list)

0.5893898972330905

In [23]:
import json
from tqdm import tqdm
with open("../../scripts/results/dev_rationale_pair_115shots.json") as f:
    rationale_pair_dev_data = json.load(f)

import pdb
bertscores = []
for da in tqdm(rationale_pair_dev_data, total=len(rationale_pair_dev_data)):
    generated_expl = da['generated_explanation']
    common_expl_list = da['common_expl_list']
    pred_expl = generated_expl.split("<extra_id_0> ")[1].split("<extra_id_1>")[0]
    list_gold_expl = common_expl_list
    #print(generated_expl)
    #print(generated_expl.split("<extra_id_0> ")[1].split("<extra_id_1>")[0])
    instance_bertscores = []
    for gold_expl in list_gold_expl: 
        score = bertscore_metric.compute(predictions=[pred_expl.lower()], references=[gold_expl.lower()], lang="en")["f1"][0]*100
        instance_bertscores.append(score)
    bertscores.append(max(instance_bertscores))
bertscore = np.mean(bertscores)
print(bertscore)
    #pdb.set_trace()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 986/986 [00:36<00:00, 26.82it/s]

90.0988023131179


In [24]:
from tqdm import tqdm
qae_list = []
score_list = []
for da in tqdm(rationale_pair_dev_data, total=len(rationale_pair_dev_data)):
    qae = "{} answer: {} explanation: {}".format(da['question'], 
                                                 da['answer'], 
                                                 da['generated_explanation'])

    scores = get_scores(
        [qae],
        'large',
        device='cuda:0',
        batch_size=1,
        verbose=False)
    score_list.append(scores[0])
    if scores[0] > 0.7 or scores[0] < 0.2:
        print("question: {}".format(da['question']))
        print("answer: {}".format(da['answer']))
        print("common_expl_list: {}".format(da['common_expl_list']))
        print("generated_explanation: {}".format(da['generated_explanation']))
        print("score: {}".format(scores[0]))

  0%|▌                                                                                                                                                                      | 3/986 [00:00<01:14, 13.27it/s]

question: Where can you likely buy many poems?
answer: book store
common_expl_list: ['A bookstore sells a variety of books, including poetry books; chains of bookstores sometimes specialize in categories such as poetry or literature.', 'A book store is a place where you can buy books, including poetry books.']
generated_explanation: <pad> <extra_id_0> Poems are often published in books, and book stores often sell poetry.<extra_id_1> </s>
score: 0.812351405620575
question: A person who yawns and paces to help pass the time is likely feeling what?
answer: boredom
common_expl_list: ['By yawning and pacing one shows signs of being bored.', 'Yawning and pacing is a sign of being bored, etc...', 'A bored person would pace and yawn to make time go by faster.']
generated_explanation: <pad> <extra_id_0> Boredom is a common cause of yawning and pacing, and it's common for people to do these things to pass the time while waiting for something to happen.<extra_id_1> </s>
score: 0.7509153485298157


  2%|███                                                                                                                                                                   | 18/986 [00:00<00:48, 19.93it/s]

question: What is a good way to carry a small dog?
answer: backpack
common_expl_list: ['Backpacks are a good way to carry things, and small dogs are things.']
generated_explanation: <pad> <extra_id_0> Small dogs are often carried in backpacks, which are designed to fit over the shoulders.<extra_id_1> </s>
score: 0.7544651031494141


  2%|███▌                                                                                                                                                                  | 21/986 [00:01<00:48, 20.10it/s]

...

question: The man was going fishing instead of work, what is he seeking?
answer: relaxation
common_expl_list: ['The man wants to make himself feel less stressed from work. Fishing is a relaxing, stress-relieving activity.', 'Fishing is a relaxing activity.', 'A man seeking relaxation does so by doing something enjoyable, e.g., taking a fishing trip.']
generated_explanation: <pad> <extra_id_0> Fishing is a relaxing activity, and if you are going fishing instead of working, you are seeking relaxation.<extra_id_1> </s>
score: 0.7439183592796326


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 979/986 [00:47<00:00, 20.40it/s]

question: He needed more information to fix it, so he consulted the what?
answer: manual
common_expl_list: ['When it comes to home repairs, one is likely to consult an online or printed search manual to learn how to fix the problem. Such publications are often hard cover (or loose leaf, but not electronically).', 'For the most part, one would need to consult a manual of instructions ("instructions manual") of a broken appliance to attempt to fix it.', "If one is trying to fix a problem with a machine, it's common to consult the manual for the machine."]
generated_explanation: <pad> <extra_id_0> Manuals contain instructions on how to fix a particular problem, and if you need more information, you can consult the manual.<extra_id_1> </s>
score: 0.7305561900138855


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 986/986 [00:48<00:00, 20.51it/s]

question: What is it called when two people in love have children?
answer: procreate
common_expl_list: ['Procreation is the act of creating new life, and is often done by people in love.', 'A main purpose of marriage, particularly in the past, was to have children. Procreation is a means to produce children.']
generated_explanation: <pad> <extra_id_0> When two people are in love, they may decide to have children. This is called procreation, and it occurs when they are in a relationship.<extra_id_1> </s>
score: 0.7677991986274719
question: What is main benefit to exercising?
answer: get in shape
common_expl_list: ['To get in shape, work out. Exercising can help you become stronger or thinner. Exercising is sometimes done to strengthen the heart or lungs, or sweat.', 'From a health perspective, exercising is healthy. It increases flexibility, circulation, improves mood, burns calories, strengthens muscles, helps us sleep better, etc.', "Exercise is a way to get in shape, which is a way t

In [25]:
np.mean(score_list)

0.6216207986648438